In [2]:
import torch
import torch.nn as nn
import math

d_model is the dimensions of the input and output token 
learned embeddings are the mapping of words to numeros

## Classes

In [4]:
class InputEmbeddings(nn.Module):
    def __init__(self, d_model:int,vocab:int):
        self.d_model=d_model
        self.vocab=vocab
        self.embeddings=nn.Embedding(vocab,d_model)
    def forward(self,x):
        return self.embeddings(x)*math.sqrt(self.d_model)

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self,d_model:int,seq_len:int,dropout:float):
        super().__init__()
        self.d_model=d_model
        self.seq_len=seq_len
        self.dropout=nn.Dropout(dropout)
        pe=torch.zeros(seq_len,d_model)# [12,512]
        position=torch.arange(0,seq_len,dtype=torch.float).unsqueeze(1) #[12] -> [12,1]
        div_term=torch.exp(torch.arange(0,d_model,2).float()*(-math.log(10000.0)/d_model))
        pe[:,0::2]=torch.sin(position*div_term)
        pe[:,1::2]=torch.cos(position*div_term)
        #pe [seq_len,d_model]
        pe=pe.unsqueeze(0)
        self.register_buffer('pe',pe)#buffer its saved with state dict and moves with model to position but cant be updated

    def forward(self,x):
        x=x+(self.pe[:,:x.shape[1],:]).requires_grad(False)
        return self.dropout(x)
    #pe[:,...,:] accesses dimensions in order first dimension get all second dimension 

In [4]:
class LayerNormalization(nn.Module):
    def __init__(self,features:int,eps:float=10**-6)->None:
        super().__init__()
        self.eps=eps
        self.alpha=nn.Parameter(torch.ones(features))
        self.bias=nn.Parameter(torch.zeros(features))

    def forward(self,x):
        mean=x.mean(dim=-1,keepdim=True)#performs means on last dim and keep the remaining dim
        std=x.std(dim=-1,keepdim=True)
        return self.alpha*(x-mean)/(std+self.eps)+self.bias#eps is so that when we divide by very small number return gets very high
    #also to prevent division by zero

In [5]:
class FeedForwardBlock(nn.Module):
    def __init__(self,d_model:int,dff:int,dropout:float):
        super().__init__()
        self.linear1=nn.Linear(d_model,dff)
        self.dropout=nn.Dropout(dropout)
        self.linear2=nn.Linear(dff,d_model)

    def forward(self,x):
        return self.linear2(self.dropout(torch.relu(self.linear1(x))))

In [6]:
class MultiHeadAttention(nn.Module):
    def __init__(self,h:int,dropout:float,d_model:int):
        super().__init__()
        self.h=h
        self.d_model=d_model
        assert d_model%h==0,"D_model isnt divisible by h"

        self.d_k=d_model//h #floor division and no_ dimensions for each head
        self.Wq=nn.Linear(d_model,d_model,bias=False)
        self.Wk=nn.Linear(d_model,d_model,bias=False)
        self.Wv=nn.Linear(d_model,d_model,bias=False)        
        self.Wo=nn.Linear(d_model,d_model,bias=False)#final weight when the qkv become one and joined to form a single head
        self.dropout=nn.Dropout(dropout)

    @staticmethod
    def attention(key,query,value,mask,dropout:nn.Dropout):
        d_k=query.shape[-1]
        #(batch,h,sqlen,d_k)->
        attention_scores=(query@key.transpose(-2,-1))/math.sqrt(d_k)#formula
        if mask is not None:
            attention_scores.masked_fill_(mask==0,-1e9)#-ve billion so when softmax it becomes zero
        attention_scores=attention_scores.softmax(dim=-1)
        if dropout is not None:
            attention_scores=dropout(attention_scores)
        return (attention_scores@value),attention_scores
    
    def forward(self,q,k,v,mask):
        query=self.Wq(q)
        key=self.Wk(q)
        value=self.Wv(q)

        query=query.view(query.shape[0],query.shape[1],self.h,self.d_k).transpose(1,2)
        key=key.view(key.shape[0],key.shape[1],self.h,self.d_k).transpose(1,2)
        value=value.view(value.shape[0],value.shape[1],self.h,self.d_k).transpose(1,2)

        x,self.attention_scores=MultiHeadAttention.attention(query,key,value,mask,self.dropout)

        x=x.transpose(1,2).continguous().view(x.shape[0],-1,self.h*self.d_k)

        return self.Wo(x)

In [7]:
class ResidualConnections(nn.Module):
    def __init__(self,features:int,dropout:float):
        super().__init__()
        self.dropout=nn.Dropout(dropout)
        self.norm=LayerNormalization()

    def forward(self,x,sublayer):
        return x+self.dropout(sublayer(self.norm(x)))

## Test

In [36]:
q=nn.Dropout(0.8)
q

Dropout(p=0.8, inplace=False)

In [8]:
seq_len=12
d_model=512

In [31]:
pe = torch.zeros(seq_len, d_model)
pe.shape
#12 rows and 512 columns

torch.Size([12, 512])

In [24]:
position = torch.arange(0, seq_len, dtype=torch.float)
position.shape

torch.Size([12])

In [30]:
positione = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1)
positione.shape

torch.Size([12, 1])

In [32]:
div=torch.arange(0, d_model, 2).float()
div

tensor([  0.,   2.,   4.,   6.,   8.,  10.,  12.,  14.,  16.,  18.,  20.,  22.,
         24.,  26.,  28.,  30.,  32.,  34.,  36.,  38.,  40.,  42.,  44.,  46.,
         48.,  50.,  52.,  54.,  56.,  58.,  60.,  62.,  64.,  66.,  68.,  70.,
         72.,  74.,  76.,  78.,  80.,  82.,  84.,  86.,  88.,  90.,  92.,  94.,
         96.,  98., 100., 102., 104., 106., 108., 110., 112., 114., 116., 118.,
        120., 122., 124., 126., 128., 130., 132., 134., 136., 138., 140., 142.,
        144., 146., 148., 150., 152., 154., 156., 158., 160., 162., 164., 166.,
        168., 170., 172., 174., 176., 178., 180., 182., 184., 186., 188., 190.,
        192., 194., 196., 198., 200., 202., 204., 206., 208., 210., 212., 214.,
        216., 218., 220., 222., 224., 226., 228., 230., 232., 234., 236., 238.,
        240., 242., 244., 246., 248., 250., 252., 254., 256., 258., 260., 262.,
        264., 266., 268., 270., 272., 274., 276., 278., 280., 282., 284., 286.,
        288., 290., 292., 294., 296., 29